In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from importlib import reload
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import Adam
from PGAgent import PGAgent
import simulation_process


Using TensorFlow backend.


In [4]:
data=pd.read_csv("long_term_maximum_CPU_usage.csv")
data1=data.fillna(0)
data1=np.array(data1)
data1=data1[:1000,:]

In [5]:
row_num=data1.shape[0]
col_num=data1.shape[1]

In [6]:
# Environment Settings
PM_num=1000;
# PM Trace Generation
PM_data=np.zeros([row_num,PM_num])
PM_vm_list=[]
PM_vm_list1=PM_vm_list.copy()
PM_vm_list2=PM_vm_list.copy()
for i in range (PM_num):
    chosen_vm_num=np.random.randint(4)
    chosen_column=list(np.random.randint(col_num,size=chosen_vm_num))
    #if i<PM_num/2:
    PM_vm_list.append(chosen_column)    
    PM_vm_list1.append(chosen_column.copy())
    PM_vm_list2.append(chosen_column.copy())
    for j in range( chosen_vm_num):
        PM_data[:,i]+=data1[:,chosen_column[j]]
   # else:
   #     PM_vm_list.append([])    
   #     PM_vm_list1.append([])
  #      PM_vm_list2.append([])

In [8]:
reload(simulation_process)
state_size=PM_num
action_size=(PM_num-1)*3+1
agent=PGAgent(state_size,action_size)
#PM_intial_workload_data, PM_initial_workload=simulation_process.PM_current_workload(data1,PM_vm_list,15,PM_num)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2998)              302798    
Total params: 412,998
Trainable params: 412,998
Non-trainable params: 0
_________________________________________________________________


C:\Users\Jimmy\Desktop\Project\Prediction_Work1\src\PGAgent.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, input_shape=(1000,), activation="relu", kernel_initializer="he_uniform")`
  model.add(Dense(50, input_shape=(self.state_size,),activation='relu', init='he_uniform'))
C:\Users\Jimmy\Desktop\Project\Prediction_Work1\src\PGAgent.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_initializer="he_uniform")`
  #model.add(Dense(100, activation='relu', init='he_uniform'))


In [9]:
reload(simulation_process)
current_time=0
PM_current_workload_data, PM_current_workload=simulation_process.PM_current_workload(data1,PM_vm_list,101+current_time,PM_num)
PM_current_workload_data1, PM_current_workload1=simulation_process.PM_current_workload(data1,PM_vm_list1,101+current_time,PM_num)
print(PM_current_workload1)
        
SLo_no_migration=len(list(filter(lambda x: x>0.95 , PM_current_workload1)))
state=simulation_process.observation_state(PM_current_workload,1)
agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []

print("initial_state:",state)

[0.1899, 0.25052, 0.12261999999999999, 0.2858, 0.15637, 0.02136, 0.13943, 0.0, 0.05072, 0.2146, 0.042789999999999995, 0, 0.32786, 0.028260000000000004, 0.11121, 0.132013, 0.09336, 0.0, 0.12268000000000001, 0.10169, 0.17115000000000002, 0.14783, 0.20226, 0.13031, 0.13178, 0.02783, 0, 0, 0.43635, 0.03943, 0.03015, 0.5733999999999999, 0.1697, 0.04393, 0.026180000000000002, 0.0278, 0.25083, 0, 0, 0.13831, 0, 0.27347, 0, 0, 0.15808, 0.0117, 0.2334, 0.2354, 0.19899999999999998, 0.04156, 0.08368, 0.1019, 0.28979999999999995, 0, 0.042910000000000004, 0.10444699999999998, 0, 0.8191599999999999, 0.12752, 0.059791000000000004, 0.12564999999999998, 0.0, 0.27420100000000003, 0.09634999999999999, 0, 0, 0, 0.03869, 0.025509999999999998, 0.041260000000000005, 0, 0.1624, 0.06731, 0.07055, 0.05033, 0, 0.03036, 0, 0, 0, 0.068738, 0.168362, 0.03992, 0.1313, 0, 0.03394, 0.23769, 0, 0.07007000000000001, 0.10808, 0.29392, 0, 0.10208, 0, 0.02121, 0, 0.009102, 0.256189, 0, 0.027160000000000004, 0.0662799999999

In [10]:
reload(simulation_process)
def RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,PM_vm_list_status,train_status):
    agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []
    mig_num=0
    for i in range(PM_num):
        NN_state=simulation_process.get_nn_state(state,i)
        if NN_state[0]==3:
            action,prob=agent.act(NN_state)
            state,reward,SLo=simulation_process.env_step(i,PM_current_workload_data,PM_current_workload,PM_vm_list,action,PM_vm_list_status)
            agent.remember(NN_state,action,prob,reward)
            mig_num+=1
   # print(agent.rewards)
    #print(agent.states)
    if(len(agent.rewards)>0 and train_status==1):
        agent.train()
    agent.states, agent.probs, agent.gradients, agent.rewards = [], [], [], []
    SLo=len(list(filter(lambda x: x>0.95 , PM_current_workload)))
    return SLo,state,mig_num

In [11]:
import pdb; 
#pdb.set_trace()
SLo_no_sum=0
SLo_sum=0
for iter in range(5):
    SLo,state,_=RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,0,1)

    print("after_state:",state)
    print("current time %d-----: SLO_no_migration:%d, SLO_RL:%d, " %(current_time,SLo_no_migration,SLo))

after_state: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2.
 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1.
 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1

after_state: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2.
 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1.
 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1

In [15]:
SLo_no_sum=0
SLo_sum=0
PM_vm_list=[]
for i in range(len(PM_vm_list1)):
    PM_vm_list.append(PM_vm_list1[i].copy())
    
for current_time in range(700):
    #re_score=score
    print ("current_time:",current_time)
    PM_current_workload_data1, PM_current_workload1=simulation_process.PM_current_workload(data1,PM_vm_list1,10+current_time,PM_num)
        #print(PM_current_workload1)    
    SLo_no_migration=len(list(filter(lambda x: x>0.95 , PM_current_workload1)))
    PM_current_workload_data, PM_current_workload=simulation_process.PM_current_workload(data1,PM_vm_list,10+current_time,PM_num)
    state=simulation_process.observation_state(PM_current_workload,1)
    #print("initial_state:",state)
    for epoch in range(1):
        SLo,state,mig_num=RL_train_epoch(PM_num,state,PM_current_workload_data,PM_current_workload,PM_vm_list,1,0)
   # print("after_state:",state)
    print("current time %d-----: SLO_no_migration:%d, SLO_RL:%d, " %(current_time,SLo_no_migration,SLo))
    SLo_sum+=SLo
    SLo_no_sum+=SLo_no_migration
    print("current time %d-----: SLO_no_sum:%d, SLO_sum:%d, " %(current_time,SLo_no_sum,SLo_sum))


current_time: 0
current time 0-----: SLO_no_migration:1, SLO_RL:0, 
current time 0-----: SLO_no_sum:1, SLO_sum:0, 
current_time: 1
current time 1-----: SLO_no_migration:0, SLO_RL:0, 
current time 1-----: SLO_no_sum:1, SLO_sum:0, 
current_time: 2
current time 2-----: SLO_no_migration:3, SLO_RL:1, 
current time 2-----: SLO_no_sum:4, SLO_sum:1, 
current_time: 3
current time 3-----: SLO_no_migration:0, SLO_RL:0, 
current time 3-----: SLO_no_sum:4, SLO_sum:1, 
current_time: 4
current time 4-----: SLO_no_migration:1, SLO_RL:0, 
current time 4-----: SLO_no_sum:5, SLO_sum:1, 
current_time: 5
current time 5-----: SLO_no_migration:1, SLO_RL:0, 
current time 5-----: SLO_no_sum:6, SLO_sum:1, 
current_time: 6
current time 6-----: SLO_no_migration:1, SLO_RL:0, 
current time 6-----: SLO_no_sum:7, SLO_sum:1, 
current_time: 7
current time 7-----: SLO_no_migration:0, SLO_RL:0, 
current time 7-----: SLO_no_sum:7, SLO_sum:1, 
current_time: 8
current time 8-----: SLO_no_migration:2, SLO_RL:0, 
current time

current time 69-----: SLO_no_migration:2, SLO_RL:0, 
current time 69-----: SLO_no_sum:93, SLO_sum:26, 
current_time: 70
current time 70-----: SLO_no_migration:1, SLO_RL:0, 
current time 70-----: SLO_no_sum:94, SLO_sum:26, 
current_time: 71
current time 71-----: SLO_no_migration:3, SLO_RL:0, 
current time 71-----: SLO_no_sum:97, SLO_sum:26, 
current_time: 72
current time 72-----: SLO_no_migration:3, SLO_RL:1, 
current time 72-----: SLO_no_sum:100, SLO_sum:27, 
current_time: 73
current time 73-----: SLO_no_migration:4, SLO_RL:0, 
current time 73-----: SLO_no_sum:104, SLO_sum:27, 
current_time: 74
current time 74-----: SLO_no_migration:0, SLO_RL:0, 
current time 74-----: SLO_no_sum:104, SLO_sum:27, 
current_time: 75
current time 75-----: SLO_no_migration:2, SLO_RL:2, 
current time 75-----: SLO_no_sum:106, SLO_sum:29, 
current_time: 76
current time 76-----: SLO_no_migration:1, SLO_RL:1, 
current time 76-----: SLO_no_sum:107, SLO_sum:30, 
current_time: 77
current time 77-----: SLO_no_migrat

current time 136-----: SLO_no_migration:1, SLO_RL:0, 
current time 136-----: SLO_no_sum:198, SLO_sum:71, 
current_time: 137
current time 137-----: SLO_no_migration:0, SLO_RL:0, 
current time 137-----: SLO_no_sum:198, SLO_sum:71, 
current_time: 138
current time 138-----: SLO_no_migration:1, SLO_RL:0, 
current time 138-----: SLO_no_sum:199, SLO_sum:71, 
current_time: 139
current time 139-----: SLO_no_migration:1, SLO_RL:0, 
current time 139-----: SLO_no_sum:200, SLO_sum:71, 
current_time: 140
current time 140-----: SLO_no_migration:0, SLO_RL:0, 
current time 140-----: SLO_no_sum:200, SLO_sum:71, 
current_time: 141
current time 141-----: SLO_no_migration:0, SLO_RL:0, 
current time 141-----: SLO_no_sum:200, SLO_sum:71, 
current_time: 142
current time 142-----: SLO_no_migration:0, SLO_RL:0, 
current time 142-----: SLO_no_sum:200, SLO_sum:71, 
current_time: 143
current time 143-----: SLO_no_migration:0, SLO_RL:0, 
current time 143-----: SLO_no_sum:200, SLO_sum:71, 
current_time: 144
current 

current time 202-----: SLO_no_migration:1, SLO_RL:0, 
current time 202-----: SLO_no_sum:307, SLO_sum:130, 
current_time: 203
current time 203-----: SLO_no_migration:1, SLO_RL:0, 
current time 203-----: SLO_no_sum:308, SLO_sum:130, 
current_time: 204
current time 204-----: SLO_no_migration:1, SLO_RL:0, 
current time 204-----: SLO_no_sum:309, SLO_sum:130, 
current_time: 205
current time 205-----: SLO_no_migration:1, SLO_RL:0, 
current time 205-----: SLO_no_sum:310, SLO_sum:130, 
current_time: 206
current time 206-----: SLO_no_migration:3, SLO_RL:3, 
current time 206-----: SLO_no_sum:313, SLO_sum:133, 
current_time: 207
current time 207-----: SLO_no_migration:1, SLO_RL:0, 
current time 207-----: SLO_no_sum:314, SLO_sum:133, 
current_time: 208
current time 208-----: SLO_no_migration:1, SLO_RL:0, 
current time 208-----: SLO_no_sum:315, SLO_sum:133, 
current_time: 209
current time 209-----: SLO_no_migration:2, SLO_RL:0, 
current time 209-----: SLO_no_sum:317, SLO_sum:133, 
current_time: 210


current time 268-----: SLO_no_migration:2, SLO_RL:0, 
current time 268-----: SLO_no_sum:375, SLO_sum:155, 
current_time: 269
current time 269-----: SLO_no_migration:0, SLO_RL:0, 
current time 269-----: SLO_no_sum:375, SLO_sum:155, 
current_time: 270
current time 270-----: SLO_no_migration:1, SLO_RL:0, 
current time 270-----: SLO_no_sum:376, SLO_sum:155, 
current_time: 271
current time 271-----: SLO_no_migration:0, SLO_RL:0, 
current time 271-----: SLO_no_sum:376, SLO_sum:155, 
current_time: 272
current time 272-----: SLO_no_migration:0, SLO_RL:0, 
current time 272-----: SLO_no_sum:376, SLO_sum:155, 
current_time: 273
current time 273-----: SLO_no_migration:2, SLO_RL:0, 
current time 273-----: SLO_no_sum:378, SLO_sum:155, 
current_time: 274
current time 274-----: SLO_no_migration:1, SLO_RL:0, 
current time 274-----: SLO_no_sum:379, SLO_sum:155, 
current_time: 275
current time 275-----: SLO_no_migration:2, SLO_RL:2, 
current time 275-----: SLO_no_sum:381, SLO_sum:157, 
current_time: 276


...

current time 400-----: SLO_no_migration:0, SLO_RL:0, 
current time 400-----: SLO_no_sum:508, SLO_sum:199, 
current_time: 401
current time 401-----: SLO_no_migration:0, SLO_RL:0, 
current time 401-----: SLO_no_sum:508, SLO_sum:199, 
current_time: 402
current time 402-----: SLO_no_migration:1, SLO_RL:0, 
current time 402-----: SLO_no_sum:509, SLO_sum:199, 
current_time: 403
current time 403-----: SLO_no_migration:0, SLO_RL:0, 
current time 403-----: SLO_no_sum:509, SLO_sum:199, 
current_time: 404
current time 404-----: SLO_no_migration:2, SLO_RL:1, 
current time 404-----: SLO_no_sum:511, SLO_sum:200, 
current_time: 405
current time 405-----: SLO_no_migration:1, SLO_RL:0, 
current time 405-----: SLO_no_sum:512, SLO_sum:200, 
current_time: 406
current time 406-----: SLO_no_migration:0, SLO_RL:0, 
current time 406-----: SLO_no_sum:512, SLO_sum:200, 
current_time: 407
current time 407-----: SLO_no_migration:0, SLO_RL:0, 
current time 407-----: SLO_no_sum:512, SLO_sum:200, 
current_time: 408


current time 466-----: SLO_no_migration:0, SLO_RL:0, 
current time 466-----: SLO_no_sum:581, SLO_sum:228, 
current_time: 467
current time 467-----: SLO_no_migration:0, SLO_RL:0, 
current time 467-----: SLO_no_sum:581, SLO_sum:228, 
current_time: 468
current time 468-----: SLO_no_migration:1, SLO_RL:0, 
current time 468-----: SLO_no_sum:582, SLO_sum:228, 
current_time: 469
current time 469-----: SLO_no_migration:0, SLO_RL:0, 
current time 469-----: SLO_no_sum:582, SLO_sum:228, 
current_time: 470
current time 470-----: SLO_no_migration:2, SLO_RL:0, 
current time 470-----: SLO_no_sum:584, SLO_sum:228, 
current_time: 471
current time 471-----: SLO_no_migration:0, SLO_RL:0, 
current time 471-----: SLO_no_sum:584, SLO_sum:228, 
current_time: 472
current time 472-----: SLO_no_migration:1, SLO_RL:0, 
current time 472-----: SLO_no_sum:585, SLO_sum:228, 
current_time: 473
current time 473-----: SLO_no_migration:0, SLO_RL:0, 
current time 473-----: SLO_no_sum:585, SLO_sum:228, 
current_time: 474


current time 532-----: SLO_no_migration:0, SLO_RL:0, 
current time 532-----: SLO_no_sum:624, SLO_sum:246, 
current_time: 533
current time 533-----: SLO_no_migration:0, SLO_RL:0, 
current time 533-----: SLO_no_sum:624, SLO_sum:246, 
current_time: 534
current time 534-----: SLO_no_migration:2, SLO_RL:1, 
current time 534-----: SLO_no_sum:626, SLO_sum:247, 
current_time: 535
current time 535-----: SLO_no_migration:2, SLO_RL:0, 
current time 535-----: SLO_no_sum:628, SLO_sum:247, 
current_time: 536
current time 536-----: SLO_no_migration:0, SLO_RL:0, 
current time 536-----: SLO_no_sum:628, SLO_sum:247, 
current_time: 537
current time 537-----: SLO_no_migration:1, SLO_RL:0, 
current time 537-----: SLO_no_sum:629, SLO_sum:247, 
current_time: 538
current time 538-----: SLO_no_migration:1, SLO_RL:0, 
current time 538-----: SLO_no_sum:630, SLO_sum:247, 
current_time: 539
current time 539-----: SLO_no_migration:0, SLO_RL:0, 
current time 539-----: SLO_no_sum:630, SLO_sum:247, 
current_time: 540


current time 598-----: SLO_no_migration:0, SLO_RL:0, 
current time 598-----: SLO_no_sum:663, SLO_sum:260, 
current_time: 599
current time 599-----: SLO_no_migration:0, SLO_RL:0, 
current time 599-----: SLO_no_sum:663, SLO_sum:260, 
current_time: 600
current time 600-----: SLO_no_migration:0, SLO_RL:0, 
current time 600-----: SLO_no_sum:663, SLO_sum:260, 
current_time: 601
current time 601-----: SLO_no_migration:0, SLO_RL:0, 
current time 601-----: SLO_no_sum:663, SLO_sum:260, 
current_time: 602
current time 602-----: SLO_no_migration:1, SLO_RL:0, 
current time 602-----: SLO_no_sum:664, SLO_sum:260, 
current_time: 603
current time 603-----: SLO_no_migration:0, SLO_RL:0, 
current time 603-----: SLO_no_sum:664, SLO_sum:260, 
current_time: 604
current time 604-----: SLO_no_migration:0, SLO_RL:0, 
current time 604-----: SLO_no_sum:664, SLO_sum:260, 
current_time: 605
current time 605-----: SLO_no_migration:0, SLO_RL:0, 
current time 605-----: SLO_no_sum:664, SLO_sum:260, 
current_time: 606


current time 664-----: SLO_no_migration:0, SLO_RL:0, 
current time 664-----: SLO_no_sum:712, SLO_sum:281, 
current_time: 665
current time 665-----: SLO_no_migration:3, SLO_RL:3, 
current time 665-----: SLO_no_sum:715, SLO_sum:284, 
current_time: 666
current time 666-----: SLO_no_migration:0, SLO_RL:0, 
current time 666-----: SLO_no_sum:715, SLO_sum:284, 
current_time: 667
current time 667-----: SLO_no_migration:1, SLO_RL:0, 
current time 667-----: SLO_no_sum:716, SLO_sum:284, 
current_time: 668
current time 668-----: SLO_no_migration:4, SLO_RL:4, 
current time 668-----: SLO_no_sum:720, SLO_sum:288, 
current_time: 669
current time 669-----: SLO_no_migration:2, SLO_RL:1, 
current time 669-----: SLO_no_sum:722, SLO_sum:289, 
current_time: 670
current time 670-----: SLO_no_migration:0, SLO_RL:0, 
current time 670-----: SLO_no_sum:722, SLO_sum:289, 
current_time: 671
current time 671-----: SLO_no_migration:1, SLO_RL:0, 
current time 671-----: SLO_no_sum:723, SLO_sum:289, 
current_time: 672


In [14]:
agent.model.save("RL_1000.h5")